In [20]:
import re
from openpyxl import load_workbook # pip install --user openpyxl
from itertools import chain

In [5]:
# path to "cable journal" excel file
# it's a MS Excel spreadsheet with a list of node connections
# in Lomonosov 2 cluster
# I am not allowed to share it.
SPREADSHEET_FILENAME = r'wire_journal_48_53.xlsx'

In [11]:
# regex for parsing rack number and other numbers from cells
# with switch names in the spreadsheet
switch_regex = re.compile(
    r"""
    КГК\.       # literally match what is written here
    (?P<rack>\d+)\.        # rack number is one or more digits, followed by dot
    (?P<second_number>\d+)\.            # then goes another non-negative integer followed by dot
    (?P<last_number>\d+)            # and another integer of the same form
    """,
    re.VERBOSE)

assert switch_regex.match("КГК.63.2.4").groups() == ("63", "2", "4")

## Parse data from the spreadsheet using openpyxl

In [15]:
def get_column_name(column):
    """Takes column as tuple as argument and returns
    its name as string"""
    return column[0].column

In [16]:
def extract_columns(worksheet, column_names):
    """
    parameters:
        worksheet -- worksheet
        column_names -- list of strings, for example
            ['A', 'C', 'E']
    returns:
        list of columns, where every column is represented
        as a tuple"""
    all_columns = worksheet.columns
    extracted_columns = [col for col in all_columns
                         if get_column_name(col) in column_names]
    assert len(extracted_columns) == len(column_names)
    return extracted_columns

In [17]:
def columns_to_tuples(columns):
    """parameters:
        columns -- columns as a tuple/list of tuples
    returns:
        list of lists/tuples, each one represents a row"""
    return [[cell.value for cell in row] for row in zip(*columns)]

In [18]:
def parse_switch_pairs(workbook):
    """Parse openpyxl workbook and extract a list of
    pairs of switches. Pair (A, B) means that swithes A
    and B are connected.
    Returns list of pairs of strings."""
    return list(chain(*[columns_to_tuples(extract_columns(worksheet, ['C', 'K']))
            for worksheet in workbook]))

In [26]:
workbook = load_workbook(SPREADSHEET_FILENAME, read_only=True)

In [27]:
#switch_connections = parse_switch_pairs(workbook)

In [31]:
worksheet = [sheet for sheet in workbook][0]
worksheet

In [33]:
worksheet.calculate_dimension()

'C1:Y160'

In [36]:
worksheet.get_squared_range()

[(<openpyxl.cell.read_only.EmptyCell at 0x7f58ce1ad1f0>,
  <openpyxl.cell.read_only.ReadOnlyCell at 0x7f58c744cb48>),
 (<openpyxl.cell.read_only.EmptyCell at 0x7f58ce1ad1f0>,
  <openpyxl.cell.read_only.ReadOnlyCell at 0x7f58c744c7d8>),
 (<openpyxl.cell.read_only.EmptyCell at 0x7f58ce1ad1f0>,
  <openpyxl.cell.read_only.ReadOnlyCell at 0x7f58c744c410>),
 (<openpyxl.cell.read_only.EmptyCell at 0x7f58ce1ad1f0>,
  <openpyxl.cell.read_only.ReadOnlyCell at 0x7f58c74392b0>),
 (<openpyxl.cell.read_only.EmptyCell at 0x7f58ce1ad1f0>,
  <openpyxl.cell.read_only.ReadOnlyCell at 0x7f58c7580db0>),
 (<openpyxl.cell.read_only.EmptyCell at 0x7f58ce1ad1f0>,
  <openpyxl.cell.read_only.ReadOnlyCell at 0x7f58ec140830>),
 (<openpyxl.cell.read_only.EmptyCell at 0x7f58ce1ad1f0>,
  <openpyxl.cell.read_only.ReadOnlyCell at 0x7f58c73f1ca8>),
 (<openpyxl.cell.read_only.EmptyCell at 0x7f58ce1ad1f0>,
  <openpyxl.cell.read_only.ReadOnlyCell at 0x7f58c7246728>),
 (<openpyxl.cell.read_only.EmptyCell at 0x7f58ce1ad1f0>,

In [32]:
help(worksheet)

Help on ReadOnlyWorksheet in module openpyxl.worksheet.read_only object:

class ReadOnlyWorksheet(builtins.object)
 |  Methods defined here:
 |  
 |  __getitem__(self, key)
 |  
 |  __init__(self, parent_workbook, title, worksheet_path, xml_source, shared_strings)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  calculate_dimension(self, force=False)
 |  
 |  get_squared_range(self, min_col, min_row, max_col, max_row)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  max_column
 |  
 |  max_row
 |  
 |  min_column
 |  
 |  min_row
 |  
 |  rows
 |  
 |  xml_source
 |      Parse xml source on demand, default to Excel archive



In [25]:
help(workbook)

Help on Workbook in module openpyxl.workbook.workbook object:

class Workbook(builtins.object)
 |  Workbook is the container for all other parts of the document.
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, key)
 |  
 |  __delitem__(self, key)
 |  
 |  __getitem__(self, key)
 |      Returns a worksheet by its name.
 |      
 |      :param name: the name of the worksheet to look for
 |      :type name: string
 |  
 |  __init__(self, write_only=False)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |  
 |  add_named_range(self, named_range)
 |      Add an existing named_range to the list of named_ranges.
 |  
 |  add_named_style(self, style)
 |      Add a named style
 |  
 |  copy_worksheet(self, from_worksheet)
 |      Copy an existing worksheet in the current workbook
 |      :warning: This function cannot copy worksheets between workbooks.
 |      worksheets can only be copied within the workbook that they belong
 |      
 